In [62]:
import pandas as pd
df=pd.read_csv('/content/llmenergy.csv')
df.head()

,Model name,model_parameters_billion,training_tokens_billion,gpu_type,num_gpus,training_hours,data_center_region,PUE,hardware_power_draw_watts_per_gpu,carbon_intensity_gco2_per_kwh,total_energy_kwh,total_carbon_footprint_kgco2e
0,GPT-3,175,300,NVIDIA V100,10000,355.2,US East (Northern Virginia),1.1,300,429,1287000,552000
1,GPT-4,1800,13000,NVIDIA H100,10000,2160,US East (Northern Virginia),1.15,700,322,16099378,5184000
2,PaLM,540,780,Google TPUv4,6144,1200,US Central (Iowa),1.11,418,430,3436000,271400
3,BLOOM,176,350,NVIDIA A100 80GB,384,2820,Europe (Finland),1.1,400,57,433196,24700
4,DeepSeek-V3,671,14800,NVIDIA H800,2048,1361,US East (South Carolina),1.15,350,560,1123598,629215


In [63]:
df.shape

(27, 12)

In [64]:
df.isnull().sum()

,0
Model name,0
model_parameters_billion,0
training_tokens_billion,0
gpu_type,0
num_gpus,0
training_hours,0
data_center_region,0
PUE,0
hardware_power_draw_watts_per_gpu,0
carbon_intensity_gco2_per_kwh,0


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Model name                         27 non-null     object
 1   model_parameters_billion           27 non-null     object
 2   training_tokens_billion            27 non-null     object
 3   gpu_type                           27 non-null     object
 4   num_gpus                           27 non-null     object
 5   training_hours                     27 non-null     object
 6   data_center_region                 27 non-null     object
 7   PUE                                27 non-null     object
 8   hardware_power_draw_watts_per_gpu  27 non-null     object
 9   carbon_intensity_gco2_per_kwh      27 non-null     object
 10  total_energy_kwh                   27 non-null     object
 11  total_carbon_footprint_kgco2e      27 non-null     object
dtypes: object(

In [66]:

numeric_cols = [
    'model_parameters_billion',
    'training_tokens_billion',
    'num_gpus',
    'training_hours',
    'PUE',
    'hardware_power_draw_watts_per_gpu',
    'carbon_intensity_gco2_per_kwh',
    'total_energy_kwh',
    'total_carbon_footprint_kgco2e'
]

#  Replace placeholders with NaN
df.replace(['Not specified', 'Not disclosed'], pd.NA, inplace=True)

# Convert numeric columns to float
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col])

# Fill NaNs with median
for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)
df['data_center_region'].fillna(df['data_center_region'].mode()[0], inplace=True)
df['gpu_type'].fillna(df['gpu_type'].mode()[0], inplace=True)

# Check result
print(df[numeric_cols].info())
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   model_parameters_billion           27 non-null     float64
 1   training_tokens_billion            27 non-null     float64
 2   num_gpus                           27 non-null     float64
 3   training_hours                     27 non-null     float64
 4   PUE                                27 non-null     float64
 5   hardware_power_draw_watts_per_gpu  27 non-null     float64
 6   carbon_intensity_gco2_per_kwh      27 non-null     float64
 7   total_energy_kwh                   27 non-null     float64
 8   total_carbon_footprint_kgco2e      27 non-null     float64
dtypes: float64(9)
memory usage: 2.0 KB
None


/tmp/ipython-input-1329585925.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-1329585925.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

,0
Model name,0
model_parameters_billion,0
training_tokens_billion,0
gpu_type,0
num_gpus,0
training_hours,0
data_center_region,0
PUE,0
hardware_power_draw_watts_per_gpu,0
carbon_intensity_gco2_per_kwh,0


In [67]:
df['carbon_per_token_kg'] = df['total_carbon_footprint_kgco2e'] / df['training_tokens_billion']

# Carbon footprint per model parameter
df['carbon_per_param_kg'] = df['total_carbon_footprint_kgco2e'] / df['model_parameters_billion']

# Energy per GPU
df['energy_per_gpu_kwh'] = df['total_energy_kwh'] / df['num_gpus']

# Energy per training hour
df['energy_per_hour_kwh'] = df['total_energy_kwh'] / df['training_hours']
df.isnull().sum()

,0
Model name,0
model_parameters_billion,0
training_tokens_billion,0
gpu_type,0
num_gpus,0
training_hours,0
data_center_region,0
PUE,0
hardware_power_draw_watts_per_gpu,0
carbon_intensity_gco2_per_kwh,0


In [73]:
# Save the cleaned and feature-enhanced dataset (with new features)
cleaned_file = "llm_energy_metrics_cleaned.csv"
df.to_csv(cleaned_file, index=False)

print(f"Cleaned dataset with features saved as '{cleaned_file}'")


Cleaned dataset with features saved as 'llm_energy_metrics_cleaned.csv'
